In [7]:
import pandas as pd
import csv

# Set the input and output file names
csv_file_name = "fact_sales_normalized.csv"
sql_file_name = "fact_sales.sql"
table_name = "fact_sales"

# Step 1: Infer column data types and generate CREATE TABLE statement
try:
    df_sample = pd.read_csv(csv_file_name, nrows=50)  # take first 50 rows for better type inference
    dtype_mapping = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(255)'
    }

    columns_sql = []
    for col, dtype in df_sample.dtypes.items():
        sql_type = dtype_mapping.get(str(dtype), 'VARCHAR(255)')
        columns_sql.append(f"    {col} {sql_type}")   # removed backticks for SQL Server/Synapse

    create_table_statement = f"CREATE TABLE {table_name} (\n" + ",\n".join(columns_sql) + "\n);\n\n"
except Exception as e:
    print(f"Error reading CSV or inferring types: {e}")
    exit()

# Step 2: Generate INSERT statements for all rows and write to file
try:
    with open(sql_file_name, 'w', encoding='utf-8') as sql_file:
        sql_file.write(create_table_statement)

        with open(csv_file_name, 'r', encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file)
            header = next(csv_reader)  # Skip header row

            for row in csv_reader:
                values = []
                for (col, dtype), value in zip(df_sample.dtypes.items(), row):
                    if value == '' or pd.isna(value):
                        values.append('NULL')
                    else:
                        if 'int' in str(dtype) or 'float' in str(dtype):
                            values.append(value)  # keep numeric unquoted
                        else:
                            safe_value = value.replace("'", "''")  # escape single quotes
                            values.append(f"'{safe_value}'")

                insert_values = ", ".join(values)
                insert_statement = f"INSERT INTO {table_name} VALUES ({insert_values});\n"
                sql_file.write(insert_statement)

    print(f"✅ Successfully created SQL file: {sql_file_name}")

except Exception as e:
    print(f"Error generating SQL statements: {e}")


✅ Successfully created SQL file: fact_sales.sql
